## **Initialize**

#### **Import to Libraries**

In [ ]:
from utils import save_tiff_with_georeference, create_dir, create_save_path
from utils.dataloader import DataLoader
from config import MODEL_CONFIG, INFERENCE_CONFIG

from keras.models import load_model
from tqdm import tqdm
from loguru import logger

import tensorflow as tf
import numpy as np
import tifffile
import cv2

import os, glob, logging

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    logger.info("GPU is available.")
else:
    logger.error("GPU is not available.")

logging.getLogger('tensorflow').disabled = True

#### **Describe to data path and load data**

In [16]:
inference_data_paths = INFERENCE_CONFIG["path"]
inference_dataloader = DataLoader(inference_data_paths)

## **Multi Model Inference**


#### **Describe model path**

In [17]:
models_path = MODEL_CONFIG["path"]
models_path = glob.glob(models_path+"/*.h5py")
models_path.sort()


Inferences for the images placed in the data folder are made using all models.

In [ ]:
for model_path in models_path:
    model = load_model(model_path)

    _,p2,_,p4 = os.path.basename(model_path).split(".")[0].split("_")
    save_main_path = f"./saves/{p2}_{p4}"

    inference_bar = tqdm(inference_dataloader)

    for img, ds, fn in inference_bar:

        is_predict = False
        save_path = create_save_path(save_main_path, fn)
        if img.shape == (1,1024,1024):
            if p2=="0512":
                predict_img = np.expand_dims(np.zeros(img.shape), axis=-1)
                for idx1 in range(2):
                    for idx2 in range(2):
                        div_predict_img = model.predict(img[:,idx1*512:(idx1+1)*512, idx2*512:(idx2+1)*512])
                        predict_img[:,idx1*512:(idx1+1)*512, idx2*512:(idx2+1)*512,:] = div_predict_img
            else:
                predict_img = model.predict(img)
            is_predict = True
        elif img.shape == (1,512,512):
            if p2=="0512":
                predict_img = model.predict(img)
                is_predict = True
            else:
                logger.error(f"The model can only infer for 1024x1024 dimensions. Your image dim={img.shape}")
        else:
            logger.error("Image sizes are not supported for inference by any model.")

        if is_predict:
            create_dir(save_main_path)
            predict_img = (predict_img.squeeze()+1)*127.5
            save_tiff_with_georeference(predict_img, save_path, ds)

        inference_bar.set_description(f"Inference of {p2}_{p4}")


## **Single Model Inference**

Inferences for the images placed in the data folder are made used models that choosen in config.yaml file.

In [ ]:
model_path = MODEL_CONFIG["single_model_path"]
if not model_path == "":
    model = load_model(model_path)

    _,p2,_,p4 = os.path.basename(model_path).split(".")[0].split("_")
    save_main_path = f"./saves/{p2}_{p4}"

    inference_bar = tqdm(inference_dataloader)

    for img, ds, fn in inference_bar:

        is_predict = False
        save_path = create_save_path(save_main_path, fn)
        if img.shape == (1,1024,1024):
            if p2=="0512":
                predict_img = np.zeros(img.shape)
                for idx1 in range(2):
                    for idx2 in range(2):
                        div_predict_img = model.predict(img[:,idx1*512:(idx1+1)*512, idx2*512:(idx2+1)*512,:])
                        predict_img[:,idx1*512:(idx1+1)*512, idx2*512:(idx2+1)*512,:] = div_predict_img
            else:
                predict_img = model.predict(img)
            is_predict = True
        elif img.shape == (1,512,512):
            if p2=="0512":
                predict_img = model.predict(img)
                is_predict = True
            else:
                logger.error(f"The model can only infer for 1024x1024 dimensions. Your image dim={img.shape}")
        else:
            logger.error("Image sizes are not supported for inference by any model.")

        if is_predict:
            create_dir(save_main_path)
            predict_img = (predict_img.squeeze()+1)*127.5
            save_tiff_with_georeference(predict_img, save_path, ds)

        inference_bar.set_description(f"Inference of {p2}_{p4}")

else:
    logger.error("Model index undefined")